In [1]:
import pandas as pd

In [2]:
path = 'D:/UZH/other/hasoc/new/rawdata/'

In [3]:
raw_datasets = pd.read_csv(path+'t1_without_ge.csv')

In [4]:
import torch
torch.cuda.empty_cache()

In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.preprocessing import LabelBinarizer

In [7]:
def tokenize_function(examples):
    return tokenizer(examples, padding="max_length", truncation=True)

In [8]:
mlb = LabelBinarizer()
y = raw_datasets['task_1'].tolist()
yt = mlb.fit_transform(y)
yt = torch.LongTensor(yt)

x = [tokenize_function(i) for i in raw_datasets['text'].tolist()]

x_train,x_test,y_train,y_test = train_test_split(x, yt , test_size=0.1, random_state=42,shuffle=True)

In [9]:
x_tr,x_val,y_tr,y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42,shuffle=True)

In [10]:
from transformers import TrainingArguments

training_args = TrainingArguments("test_trainer")

In [11]:
def create_datamap(x, y):
    for j,i in enumerate(x):
        i['attention_mask']= torch.IntTensor(i['attention_mask'])
        i['input_ids']= torch.IntTensor(i['input_ids'])
        i['token_type_ids']= torch.IntTensor(i['token_type_ids'])
        i['labels'] = y[j]
    return x

In [12]:
train_dataset = create_datamap(x_tr, y_tr)
eval_dataset = create_datamap(x_val, y_val)

In [13]:
from torch.utils.data import DataLoader
batch_size = 1
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
eval_dataloader = DataLoader(eval_dataset, batch_size=batch_size)

In [14]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [15]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

In [16]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

In [17]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [18]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
progress_bar.close()

100%|██████████| 74961/74961 [1:39:01<00:00, 12.62it/s]


In [19]:
import numpy as np
from datasets import load_metric

In [25]:
metric= load_metric("accuracy")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"][0])

metric.compute()

{'accuracy': 0.6313430446614375}

In [26]:
model.save_pretrained("hasoc")